## HW6 - Q3

In [11]:
from pulp import *
import pandas as pd

# weather
start_weather = "normal"
weather = ["normal", "cold", "very cold"]

# demand
demand = {"normal" : 100, "cold" : 150, "very cold" : 180}

# gas cost
gas_cost = {"normal" : 5, "cold" : 6, "very cold" : 7.5}

# storage cost
storage_cost = 1

# transition probabilities
prob_t1_t2 = {"normal" : 1/3, "cold" : 1/3, "very cold" : 1/3}
prob_t2_t3 = {"normal" : {"normal" : 1/3, "cold" : 1/3, "very cold" : 1/3}, 
            "cold" : {"normal" : 1/3, "cold" : 1/3, "very cold" : 1/3}, 
            "very cold" : {"normal" : 1/3, "cold" : 1/3, "very cold" : 1/3}}

In [12]:
def create_vars(var):
    var_list = [var + start_weather]
    var_list = var_list + [var + start_weather + "_" + x for x in weather]
    var_list = var_list + [var + start_weather + "_" + x + "_" + y for x in weather for y in weather]
    return var_list

# creating variable lists
buy_to_use = create_vars("use_")
buy_to_store = create_vars("store_")
storage = create_vars("storage_")


# Create the 'prob' variable to contain the problem data
prob = LpProblem("Gas Problem", LpMinimize)


# Create the decision Variables
vars = {p : LpVariable("buy_to_use_{}".format(p), lowBound = 0, upBound = None, cat = "Integer") for p in buy_to_use}
vars.update({p : LpVariable("buy_to_store_{}".format(p), lowBound = 0, upBound = None, cat = "Integer") for p in buy_to_store})
vars.update({p : LpVariable("storage_{}".format(p), lowBound = 0, upBound = None, cat = "Integer") for p in storage})


# The objective function is added to 'prob' first
# the objective function is the expected cost

# cost of buying to use
buy_to_use_cost = [vars[x]*gas_cost[x.split("_")[-1]]*prob_t1_t2[x.split("_")[-1]] if len(x.split("_")) == 3 \
                   else vars[x]*gas_cost[x.split("_")[-1]]*prob_t1_t2[x.split("_")[-2]]*prob_t2_t3[x.split("_")[-2]][x.split("_")[-1]] if len(x.split("_")) == 4 \
                   else vars[x]*gas_cost[x.split("_")[-1]] for x in buy_to_use]

# cost of buying to use
buy_to_store_cost = [vars[x]*gas_cost[x.split("_")[-1]]*prob_t1_t2[x.split("_")[-1]] if len(x.split("_")) == 3 \
                     else vars[x]*gas_cost[x.split("_")[-1]]*prob_t1_t2[x.split("_")[-2]]*prob_t2_t3[x.split("_")[-2]][x.split("_")[-1]] if len(x.split("_")) == 4 \
                     else vars[x]*gas_cost[x.split("_")[-1]] for x in buy_to_store]

# cost of storage
storage_cost = [vars[x]*storage_cost*prob_t1_t2[x.split("_")[-1]] if len(x.split("_")) == 3 \
                else vars[x]*storage_cost*prob_t1_t2[x.split("_")[-2]]*prob_t2_t3[x.split("_")[-2]][x.split("_")[-1]] if len(x.split("_")) == 4 \
                else vars[x]*storage_cost for x in storage]

# adding the expected cost
prob += lpSum(buy_to_use_cost + buy_to_store_cost + storage_cost), "Total Expected Cost"


# constraints on storage
for x in buy_to_store:
    splits = x.split("_")
    if len(splits) == 2:
        prob += vars["storage_normal"] == vars[x]
    elif len(splits) == 3:
        prob += vars["storage_normal_" + splits[-1]] == vars[x] + vars["_".join(["use"] + splits[1:])] + vars["storage_normal"] - demand[splits[-1]]
    elif len(splits) == 4:
        prob += vars["_".join(["storage", "normal"] + splits[-2:])] == vars[x] + vars["_".join(["use"] + splits[1:])] + vars["storage_normal_" + splits[-2]] - demand[splits[-1]]
    

# constraints for availability for demand being greater than demand
for x in buy_to_use:
    if len(x.split("_")) == 2:
        prob += vars[x] >= demand[x.split("_")[-1]]
    elif len(x.split("_")) == 3:
        prob += vars["storage_normal"] + vars[x] >= demand[x.split("_")[-1]]
    elif len(x.split("_")) == 4:
        prob += vars["storage_normal_" + x.split("_")[-2]] + vars[x] >= demand[x.split("_")[-1]]

# The problem data is written to an .lp file
prob.writeLP("GasProblem.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

# The optimised objective function value is printed to the screen
print("Total Expected Cost = ", value(prob.objective))

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

#for constraint in prob.constraints:
        #print(prob.constraints[constraint].name, prob.constraints[constraint].value())
        #print(prob.constraints[constraint].name, prob.constraints[constraint].constant)

Status: Optimal
Total Expected Cost =  2302.777777777778
buy_to_store_store_normal = 150.0
buy_to_store_store_normal_cold = 0.0
buy_to_store_store_normal_cold_cold = 0.0
buy_to_store_store_normal_cold_normal = 0.0
buy_to_store_store_normal_cold_very_cold = 0.0
buy_to_store_store_normal_normal = 50.0
buy_to_store_store_normal_normal_cold = 0.0
buy_to_store_store_normal_normal_normal = 0.0
buy_to_store_store_normal_normal_very_cold = 0.0
buy_to_store_store_normal_very_cold = 0.0
buy_to_store_store_normal_very_cold_cold = 0.0
buy_to_store_store_normal_very_cold_normal = 0.0
buy_to_store_store_normal_very_cold_very_cold = 0.0
buy_to_use_use_normal = 100.0
buy_to_use_use_normal_cold = 0.0
buy_to_use_use_normal_cold_cold = 150.0
buy_to_use_use_normal_cold_normal = 100.0
buy_to_use_use_normal_cold_very_cold = 180.0
buy_to_use_use_normal_normal = 0.0
buy_to_use_use_normal_normal_cold = 50.0
buy_to_use_use_normal_normal_normal = 0.0
buy_to_use_use_normal_normal_very_cold = 80.0
buy_to_use_use_n